<a href="https://colab.research.google.com/github/gustavox0/Tecnicas_ML/blob/main/M6_AG1_Grupo17_VP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Contexto

IMDb es una de las mayores bases de datos de películas del mundo. A través de su sitio web es posible acceder a títulos, sinopsis, géneros, fechas de estreno y puntuaciones de millones de películas. En esta práctica, no se utilizará un dataset preexistente, sino que el alumno deberá **construir su propio dataset mediante web scraping**.

Se pide:

Con los datos recogidos, se entrenarán tres modelos supervisados de clasificación, todos ellos a partir del texto de la película: el título, la sinopsis o ambos.

  - Utilizar la página de exploración de géneros de IMDb: https://www.imdb.com/search/title/?explore=genres para extraer los siguientes datos para cada película:
    - Título
    - Sinopsis (texto resumen)
    - Fecha de estreno
    - Género principal
    - Puntuación media de IMDb.

Se deben obtener al menos 5 géneros distintos.

El número de películas por género queda a elección del alumno, pero se valorará reflexionar (y/o experimentar) sobre como influye el tamaño del dataset en la calidad del modelo.

# **Carga y exploración inicial del dataset**


**Creacion del DATASET**

Para la búsqueda y descarga de información, recurrimos a una estrategia de webscrapping. Para ello, utilizamos la API de OMDb.
Esta API se puede utilizar en una versión gratuita que permite la descarga de 1000 registros por día.

In [1]:
import requests
import pandas as pd
import time
from random import uniform

# API Key de OMDb
OMDB_API_KEY = "ff59522d" #Si quieres generar un API key y correr el código puedes hacerlo aquí https://www.omdbapi.com/apikey.aspx

# Géneros que queremos obtener
GENRES = ['Action', 'Comedy', 'Drama', 'Horror', 'Sci-Fi']
MOVIES_PER_GENRE = 200
SEARCH_TERMS = ['a', 'e', 'the', 'life', 'love', 'dark', 'war', 'night'] #Dado que IMDb no permite búsqueda por género, ampliamos la busqueda con estos términos
OUTPUT_FILE = "dataset_movies.csv" #guardar datos en csv

def fetch_movie_details(imdb_id): #Funcion para consultar los detalles de la película por su ID
    url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&i={imdb_id}&plot=short"
    try:
        response = requests.get(url)
        return response.json()
    except Exception as e:
        print(f" Error al obtener detalles de {imdb_id}: {e}")
        return None

def get_movies_by_genre(target_genre, max_movies=200): #Funcion para buscar película por género
    collected = [] #para guardar listado de películas encontradas
    seen_ids = set()

    for term in SEARCH_TERMS:
        for page in range(1, 11):  # hasta 10 páginas por término de búsqueda
            if len(collected) >= max_movies: #si alcanza el número de pelis (200) de detiene la función
                return collected

            url = f"http://www.omdbapi.com/?apikey={OMDB_API_KEY}&s={term}&type=movie&page={page}" #llamada a API
            try:
                res = requests.get(url)
                search_data = res.json()
            except Exception as e:
                print(f" Error al buscar películas con término '{term}' (página {page}): {e}")
                break

            if search_data.get('Response') != 'True':
                break  # sin más resultados válidos

            for item in search_data.get('Search', []):
                imdb_id = item.get('imdbID')
                if not imdb_id or imdb_id in seen_ids:
                    continue

                movie_data = fetch_movie_details(imdb_id) #llama a función de detalles de película
                if not movie_data:
                    continue

                genres = movie_data.get('Genre', '') #filtra a películas de los géneros buscados.
                if target_genre not in genres:
                    continue

                collected.append({                     #guarda la info deseada de cada película
                    'Título': movie_data.get('Title'),
                    'Sinopsis': movie_data.get('Plot'),
                    'Fecha de estreno': movie_data.get('Released'),
                    'Género principal': target_genre,
                    'Puntuación IMDb': movie_data.get('imdbRating')
                })

                seen_ids.add(imdb_id)
                print(f"[{target_genre}] {len(collected)}. {movie_data.get('Title')}")

                if len(collected) >= max_movies:
                    break

                time.sleep(uniform(0.4, 0.8))  # espera entre consultas para evitar bloqueo de la página

            time.sleep(uniform(0.8, 1.5))

    return collected

# Ejecutar la búsqueda por todos los géneros
all_movies = []
for genre in GENRES:
    print(f"\n Buscando películas del género: {genre}")
    movies = get_movies_by_genre(genre, MOVIES_PER_GENRE)
    all_movies.extend(movies)

# Guardar resultados en CSV
df = pd.DataFrame(all_movies)
df.to_csv(OUTPUT_FILE, index=False)
print(f"\n {len(df)} películas guardadas en {OUTPUT_FILE}")



 Buscando películas del género: Action
[Action] 1. The Life Aquatic with Steve Zissou
[Action] 2. Bad Boys for Life
[Action] 3. Lara Croft: Tomb Raider - The Cradle of Life
[Action] 4. Proof of Life
[Action] 5. A Bittersweet Life
[Action] 6. Term Life
[Action] 7. Thor: Love and Thunder
[Action] 8. Love and Monsters
[Action] 9. From Russia with Love
[Action] 10. From Paris with Love
[Action] 11. Love Exposure
[Action] 12. I Love Trouble
[Action] 13. Love Hurts
[Action] 14. The Dark Knight
[Action] 15. The Dark Knight Rises
[Action] 16. Thor: The Dark World
[Action] 17. Transformers: Dark of the Moon
[Action] 18. X-Men: Dark Phoenix
[Action] 19. Terminator: Dark Fate
[Action] 20. The Dark Tower
[Action] 21. Batman: The Dark Knight Returns, Part 1
[Action] 22. Under Siege 2: Dark Territory
[Action] 23. Alone in the Dark
[Action] 24. Batman: The Dark Knight Returns, Part 2
[Action] 25. Hold the Dark
[Action] 26. Justice League Dark
[Action] 27. Justice League Dark: Apokolips War
[Action] 

La busqueda anterior genera el archivo csv "dataset_movies.csv". Por el límite diario que permite la API de OMDb de manera gratuita (1000 registros), a partir de acá guardamos el archivo generado y trabajemos con el para no realizar el webscrapping cada vez.

In [3]:
# Base generada mediante webscrapping
dataset = pd.read_csv("https://raw.githubusercontent.com/gustavox0/Tecnicas_ML/refs/heads/main/dataset_movies.csv")
dataset.head()


,Título,Sinopsis,Fecha de estreno,Género principal,Puntuación IMDb
0,The Life Aquatic with Steve Zissou,With a plan to exact revenge on a legendary sh...,25 Dec 2004,Action,7.2
1,Bad Boys for Life,Miami detectives Mike Lowrey and Marcus Burnet...,17 Jan 2020,Action,6.5
2,Lara Croft: Tomb Raider - The Cradle of Life,Lara Croft sets out on a quest to prevent Pand...,25 Jul 2003,Action,5.5
3,Proof of Life,Alice hires a professional negotiator to obtai...,08 Dec 2000,Action,6.3
4,A Bittersweet Life,Things go wrong for a high ranking mobster whe...,01 Apr 2005,Action,7.5


In [14]:
#Resumen de la base
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579 entries, 0 to 578
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Título            579 non-null    object 
 1   Sinopsis          579 non-null    object 
 2   Fecha de estreno  578 non-null    object 
 3   Género principal  579 non-null    object 
 4   Puntuación IMDb   577 non-null    float64
dtypes: float64(1), object(4)
memory usage: 22.7+ KB
None


Como puede observarse, el dataset contiene la información de 579 películas y los atributos Título, Sinópsis, Fecha de estreno, Género de película y
Puntuación IMDb.

In [13]:
# Películas por género
print(round(dataset['Género principal'].value_counts(normalize=True) * 100,2))

Género principal
Drama     34.54
Comedy    29.53
Action    17.44
Horror    13.30
Sci-Fi     5.18
Name: proportion, dtype: float64


Sin embargo, el dataset se encuentra desbalanceado. El género con menos películas es el Sci-Fi (5.18%) y el que tiene más información es el de Drama (34.45%).  

# **Preprocesamiento**

  - Limpiar el texto: minúsculas, eliminar símbolos, stopwords, etc.
  - Aplicar vectorización con TF-IDF, CountVectorizer o embeddings.
  - Crear las siguientes variables objetivo:
    - Género (multiclase)
    - Fecha de estreno (agrupar por décadas, por ejemplo: 1990s, 2000s, 2010s, 2020s...)
    - Puntuación (variable continua)

# **Modelado**

  - Entrenar tres modelos de clasificación, uno por cada variable objetivo.
  - Probar al menos dos algoritmos por modelo.
  - Comparar sus métricas (accuracy, F1-score, matriz de confusión...).
  - Justificar la elección del algoritmo final.

# **Visualización**

  - Comentar los resultados obtenidos.
  - ¿Qué variable fue más fácil de predecir? ¿El título o la sinopsis aporta más información?
  - ¿Cuántas películas se necesitan por clase para que el modelo generalice bien?